In [110]:
import pandas as pd
import yfinance as yf
import numpy as np
import datetime as dt
import copy
import matplotlib.pyplot as plt
from yahooquery import Ticker

In [111]:
#importando base
df_ibrx = pd.read_excel(r'C:\Users\drsan\OneDrive\Área de Trabalho\Python\ETFs_Onshore.xlsx',sheet_name = 'IBRX100')
df_ibrx.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Setor,Subsetor,Segmento
0,ABEV3,AMBEV S/A,ON,4.354.228.928,2936.0,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
1,ALPA4,ALPARGATAS,PN N1,187.007.704,328.0,Consumo Cíclico,"Tecidos, Vestuário e Calçados",Calçados
2,ALSO3,ALIANSCSONAE,ON NM,127.374.163,186.0,Financeiro,Exploração de Imóveis,Exploração de Imóveis
3,AZUL4,AZUL,PN N2,317.471.474,363.0,Bens Industriais,Transporte,Transporte Aéreo
4,B3SA3,B3,ON NM,2.046.021.644,6301.0,Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos


In [112]:
# dando aquele tapa
df = df_ibrx
df = pd.DataFrame(df) 
df = df.dropna()
df['Part. (%)'] /= 100000
df.rename(columns={'Código\xa0':'Codigo'}, inplace = True)
df['Codigo'] = df['Codigo'] + ".SA"
tickers = df['Codigo']

In [113]:
start = dt.date.today() + dt.timedelta(1) - dt.timedelta(59)
end = dt.date.today() + dt.timedelta(1)
prices = {}
for t in tickers:
    try:
        prices[t] = yf.download(t,start,end,interval='1d')  
    except: 
        continue
    prices[t].dropna(how='all', inplace = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [114]:
prices2 = copy.deepcopy(prices)
for t in tickers:
    try:
        prices2[t]['roll_avg_5d'] = prices2[t]['Close'].rolling(5).mean()
        prices2[t]['roll_avg_10d'] = prices2[t]['Close'].rolling(10).mean()
        prices2[t]['roll_avg_20d'] = prices2[t]['Close'].rolling(20).mean()

        prices2[t]['roll_exp_5d'] = prices2[t]['Close'].ewm(span=5).mean()
        prices2[t]['roll_exp_10d'] = prices2[t]['Close'].ewm(span=10).mean()
        prices2[t]['roll_exp_20d'] = prices2[t]['Close'].ewm(span=20).mean()

        prices2[t]['roll_max_5d'] = prices2[t]['Close'].rolling(5).max()
        prices2[t]['roll_max_10d'] = prices2[t]['Close'].rolling(10).max()
        prices2[t]['roll_max_20d'] = prices2[t]['Close'].rolling(20).max()

        prices2[t]['roll_min_5d'] = prices2[t]['Close'].rolling(5).min()
        prices2[t]['roll_min_10d'] = prices2[t]['Close'].rolling(10).min()
        prices2[t]['roll_min_20d'] = prices2[t]['Close'].rolling(20).min()

        prices2[t].dropna(inplace=True)
        
    except:
        continue

In [115]:
prices3 = pd.DataFrame(columns=['ticker','Setor', 'Subsetor', 'Segmento', 'Close', 'Close_D-1', 'roll_avg_5d',
                                'roll_avg_20d', 'roll_exp_5d','roll_exp_20d','roll_max_5d','roll_max_20d',
                                'roll_min_5d','roll_min_20d'])
for t in range(len(tickers)):
    try:
        prices3.loc[t,'ticker'] = str(tickers[t]).replace('.SA','')
        prices3.iloc[t,1] = df[df['Codigo'] == tickers[t]]['Setor'].to_list()[0]
        prices3.iloc[t,2] = df[df['Codigo'] == tickers[t]]['Subsetor'].to_list()[0]
        prices3.iloc[t,3] = df[df['Codigo'] == tickers[t]]['Segmento'].to_list()[0]
        prices3.iloc[t,4] = prices2[tickers[t]]['Close'][-1]
        prices3.iloc[t,5] = prices2[tickers[t]]['Close'][-2]

        prices3.iloc[t,6] = prices2[tickers[t]]['roll_avg_5d'][-1]
        prices3.iloc[t,7] = prices2[tickers[t]]['roll_avg_20d'][-1]

        prices3.iloc[t,8] = prices2[tickers[t]]['roll_exp_5d'][-1]
        prices3.iloc[t,9] = prices2[tickers[t]]['roll_exp_20d'][-1]    

        prices3.iloc[t,10] = prices2[tickers[t]]['roll_max_5d'][-1]
        prices3.iloc[t,11] = prices2[tickers[t]]['roll_max_20d'][-1]    

        prices3.iloc[t,12] = prices2[tickers[t]]['roll_min_5d'][-1]
        prices3.iloc[t,13] = prices2[tickers[t]]['roll_min_20d'][-1]
    
    except:
        continue
    
# stochastic
prices3['stochastic'] = (prices3['Close'] - prices3['roll_min_20d']) / (prices3['roll_max_20d'] - prices3['roll_min_20d']) * 100
    
#variacoes
prices3['R1'] = prices3['Close'] / prices3['Close_D-1'] - 1
prices3['R2'] = prices3['Close'] / prices3['roll_exp_5d'] - 1
prices3['R3'] = prices3['Close'] / prices3['roll_avg_20d'] - 1
prices3['R4'] = (prices3['roll_max_20d'] + prices3['roll_min_20d']) / prices3['Close'] - 2

#ranking
prices3['Ranking_1'] = prices3['R1'].rank(ascending = True)
prices3['Ranking_2'] = prices3['R2'].rank(ascending = False)
prices3['Ranking_3'] = prices3['R3'].rank(ascending = True)
prices3['Ranking_4'] = prices3['R4'].rank(ascending= False)
prices3['Soma_total'] = prices3['Ranking_1'] + prices3['Ranking_2']*2 + prices3['Ranking_3']*2 + prices3['Ranking_4']
prices3['Ranking_Final'] = prices3['Soma_total'].rank(ascending= True)


In [116]:
# brincando no excel
#prices3.to_excel(r'C:\Users\drsan\OneDrive\Área de Trabalho\Python\IBRX100.xlsx',sheet_name = 'IBRX100')

In [117]:
#nova carteira

#com stocastico
prices3[(prices3['R2']>0)&(prices3['R3']<0)&(prices3['stochastic']<30)].sort_values(['Ranking_Final'], ascending = True)

#sem stocastico
#prices3[(prices3['R2']>0)&(prices3['R3']<0)].sort_values(['Ranking_Final'], ascending = True).head(15)

,ticker,Setor,Subsetor,Segmento,Close,Close_D-1,roll_avg_5d,roll_avg_20d,roll_exp_5d,roll_exp_20d,...,R1,R2,R3,R4,Ranking_1,Ranking_2,Ranking_3,Ranking_4,Soma_total,Ranking_Final
72,PCAR3,Consumo não Cíclico,Comércio e Distribuição,Produtos de Limpeza,64.92,64.73,64.886,66.2465,64.8743,66.5073,...,0.00293519,0.000704516,-0.0200237,0.0779422,36.0,31.0,40.0,32.0,210.0,4.0
55,ITUB4,Financeiro,Intermediários Financeiros,Bancos,24.22,23.79,24.174,25.015,24.1669,24.9162,...,0.0180748,0.00219678,-0.031781,0.108588,96.0,27.0,29.0,16.0,224.0,10.0
54,ITSA4,Financeiro,Intermediários Financeiros,Bancos,9.72,9.61,9.728,10.012,9.71861,9.94806,...,0.0114465,0.0001433,-0.029165,0.0895061,81.0,33.0,31.0,24.0,233.0,14.0
35,EGIE3,Utilidade Pública,Energia Elétrica,Energia Elétrica,43.4,43.11,43.076,44.2175,43.1973,43.6825,...,0.006727,0.00469227,-0.0184881,0.0548386,57.0,22.0,46.0,41.0,234.0,17.0
11,BPAC11,Financeiro,Intermediários Financeiros,Bancos,81.97,80.65,81.968,85.007,81.9604,83.9811,...,0.016367,0.000117709,-0.0357264,0.05868,93.0,34.0,25.0,38.0,249.0,23.0
42,EQTL3,Utilidade Pública,Energia Elétrica,Energia Elétrica,23.96,23.78,23.888,24.2165,23.8882,24.057,...,0.00756932,0.00300388,-0.010592,0.0388148,63.0,25.0,57.0,49.0,276.0,36.0
7,BBDC4,Financeiro,Intermediários Financeiros,Bancos,21.01,20.73,20.956,21.381,20.9499,21.3582,...,0.013507,0.00286976,-0.0173519,0.0490243,90.0,26.0,49.0,46.0,286.0,41.5
48,GOLL4,Bens Industriais,Transporte,Transporte Aéreo,18.02,17.55,17.942,18.069,17.9251,18.2172,...,0.0267807,0.00529541,-0.00271179,0.0560487,100.0,21.0,63.0,39.0,307.0,53.0


In [118]:
#carteira ativa
tickers_atual = ['LIGT3',
'PCAR3',
'SMLS3',
'BBDC4',
'NEOE3',
'CMIG4',
'CSMG3',
'EGIE3',
'SAPR11',
'BTOW3']

#tickers_atual = [n + '.SA' for n in tickers_atual]

#filtro sobre quais ficam
prices3[(prices3['R2']>0)&(prices3['R3']<0)&(prices3['stochastic']<50)&(prices3['ticker'].isin(tickers_atual))].sort_values(['Ranking_Final'], ascending = True)[['ticker','stochastic']]

,ticker,stochastic
60,LIGT3,30.5405
72,PCAR3,14.3662
35,EGIE3,18.5185
7,BBDC4,21.8579
85,SMLS3,37.788


In [119]:
#carteira ativa
tickers_atual2 = ['SMLS3',
'LIGT3',
'EGIE3',
'HYPE3',
'SAPR11',
'PCAR3',
'CCRO3',
'BBDC4',
'NEOE3']

#tickers_atual = [n + '.SA' for n in tickers_atual]

#filtro sobre quais ficam
prices3[(prices3['R2']>0)&(prices3['R3']<0)&(prices3['stochastic']<50)&(prices3['ticker'].isin(tickers_atual2))].sort_values(['Ranking_Final'], ascending = True)[['ticker','stochastic']]

,ticker,stochastic
60,LIGT3,30.5405
72,PCAR3,14.3662
35,EGIE3,18.5185
7,BBDC4,21.8579
85,SMLS3,37.788
